###### Credit EDA 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

<span style="color:Brown">Analysis of Application_data : the data that shows if Client has payment difficulties </span>

In [ ]:
# Filtering out the warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#displaying all the rows and columns
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [ ]:
#importing data from CSV file into pandas dataframe
application_data =pd.read_csv("/kaggle/input/credit-card/application_data.csv")

In [ ]:
application_data.shape

In [ ]:
application_data.info()

In [ ]:
application_data.head()

In [ ]:
application_data.describe()

In [ ]:
#dropping the columns that store the Normalized information about building where the client lives
MODE_cols = [col for col in application_data.columns if '_MODE' in col]
application_data.drop(columns = MODE_cols,inplace=True)
MEDI_cols = [col for col in application_data.columns if '_MEDI' in col]
application_data.drop(columns = MEDI_cols,inplace=True)
AVG_cols = [col for col in application_data.columns if '_AVG' in col]
application_data.drop(columns = AVG_cols,inplace=True)

In [ ]:
#Find all the columns starting with name FLAG_DOCUMENT , as these are just FLAGS indicating document submission and can be dropped
temp=application_data.filter(regex='^FLAG_DOCUMENT',axis=1)

In [ ]:
#drop the FLAG_DOCUMENT Columns the column names are stored in temp Series
application_data.drop(columns=temp.columns,inplace=True)

In [ ]:
#These columns 'FLAG_MOBIL','FLAG_EMP_PHONE','FLAG_WORK_PHONE','FLAG_CONT_MOBILE','FLAG_PHONE','FLAG_EMAIL' 
#are FLAGS which need not be used in analysis so drop them
application_data.drop(columns=['FLAG_MOBIL',
                           'FLAG_EMP_PHONE',
                           'FLAG_WORK_PHONE',
                           'FLAG_CONT_MOBILE',
                           'FLAG_PHONE',
                            'FLAG_EMAIL'],inplace=True)

In [ ]:
application_data.isnull().sum().sort_values(ascending=False)

In [ ]:
#Percentage of null values in each column
col_with_nullvalues=application_data.isnull().sum()/len(application_data)*100
col_with_nullvalues.sort_values(ascending=False)

In [ ]:
# List the columns having more than 50% missing values
nullvalues_50=col_with_nullvalues[col_with_nullvalues.values>50.0].sort_values(ascending=False)
nullvalues_50

In [ ]:
application_data.shape

In [ ]:
#drop the columns with 60 % or more null values
col_with_nullvalues = list(col_with_nullvalues[col_with_nullvalues.values>=60.0].index)
application_data.drop(labels=col_with_nullvalues,axis=1,inplace=True)
print(len(col_with_nullvalues))

In [ ]:
# Checking the columns having less null percentage
application_data.isnull().sum()/len(application_data)*100

<span style="color:Blue">Insights: EXT_SOURCE_1 is not dropped as we might need it for analysis</span>

In [ ]:
# columns with int64 data types have 2 unique values and are flags
#these columns cannot be used in bivariate analysis as they are not categorical.
application_data.select_dtypes('int64').apply(pd.Series.nunique, axis = 0)

In [ ]:
#Let us drop these columns as they have high null values and also their relevance in the analysis is insignificance, 
#we can may be keep one such column 'AMT_REQ_CREDIT_BUREAU_QRT' which could be helpful.
application_data.drop(columns = ['AMT_REQ_CREDIT_BUREAU_YEAR',
                             'AMT_REQ_CREDIT_BUREAU_MON',
                             'AMT_REQ_CREDIT_BUREAU_WEEK',
                             'AMT_REQ_CREDIT_BUREAU_DAY',
                             'AMT_REQ_CREDIT_BUREAU_HOUR'],inplace=True)

In [ ]:
# we need to drop other columns which we dont need
application_data.drop(columns = ['REG_REGION_NOT_LIVE_REGION',
                             'REG_REGION_NOT_WORK_REGION',
                             'LIVE_REGION_NOT_WORK_REGION',
                             'REG_CITY_NOT_LIVE_CITY',
                             'REG_CITY_NOT_WORK_CITY',
                            'LIVE_CITY_NOT_WORK_CITY'],inplace=True)

In [ ]:
#checking current shape of application_data
application_data.shape

In [ ]:
application_data.info()

In [ ]:
#Sanity Check for columns and converting the negative values to positive values
application_data['DAYS_BIRTH'] = abs(application_data['DAYS_BIRTH'])
application_data['DAYS_ID_PUBLISH'] = abs(application_data['DAYS_ID_PUBLISH'])
application_data['DAYS_EMPLOYED'] = abs(application_data['DAYS_EMPLOYED'])
application_data['DAYS_REGISTRATION'] = abs(application_data['DAYS_REGISTRATION'])

In [ ]:
# Get the count,size and unique values in each column of application data
application_data.agg(['count','size','nunique'])

<span style="color:Blue"> <b>Observation : </b>A lot of "int" columns have 2 unique values. These are flags or Flag type varibles(not useful for bivariate analysis) These can be converted to Yes/No values for categorical analysis.</span>

###### Starting with Univariate Analysis

In [ ]:
#Checking the distribution of target variable
defaulters=application_data[application_data.TARGET==1]
nondefaulters=application_data[application_data.TARGET==0]

In [ ]:
#checking percentage of defaulters
percentage_defaulters=(len(defaulters)*100)/len(application_data)
percentage_defaulters

<span style="color:Blue"><b>Observation:</b>The Percentage of people who have NOT paid their loan is: 8.07 %</span>

In [ ]:
#checking percentage of non-defaulters
percentage_nondefaulters=(len(nondefaulters)*100)/len(application_data)
percentage_nondefaulters

<span style="color:Blue"><b>Observation:</b>The Percentage of people who have paid their loan is: 91.93 %</span>

In [ ]:
sns.countplot(application_data.TARGET)
plt.xlabel("TARGET")
plt.ylabel("Number of TARGET")
plt.title("Distribution of TARGET Variable")
plt.show()

<span style="color:Blue"><b>Observation: </b>this is an imbalanced dataset. There are far more loans that were repaid on time than loans that were not repaid.</span>

In [ ]:
#Looking at the data types of the columns considered for analysis
application_data.select_dtypes('object').columns

In [ ]:
application_data.select_dtypes('float64').columns

In [ ]:
application_data.select_dtypes('int64').columns

###### Categorical Analysis

In [ ]:
defaulters=application_data[application_data.TARGET==1]
nondefaulters=application_data[application_data.TARGET==0]

In [ ]:
#function for plotting categories based on defaulters and non-defaulters
def biplot(data, var,label_rotation):
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(18,10))
    s1=sns.barplot(ax=ax1,x=defaulters[var].value_counts().index, data=defaulters, y= 100.* defaulters[var].value_counts(normalize=True))
    if(label_rotation):
        s1.set_xticklabels(s1.get_xticklabels(),rotation=90,fontsize=12)
    ax1.set_title('Distribution of '+ '%s' %var +' - Defaulters', fontsize=15)
    ax1.set_xlabel('%s' %var,fontsize=15)
    ax1.set_ylabel("% of Loans",fontsize=15)
   
    s2=sns.barplot(ax=ax2,x=nondefaulters[var].value_counts().index, data=nondefaulters, y= 100.* nondefaulters[var].value_counts(normalize=True))
    if(label_rotation):
        s2.set_xticklabels(s2.get_xticklabels(),rotation=90,fontsize=12)
    ax2.set_xlabel('%s' %var, fontsize=15)
    ax2.set_ylabel("% of Loans", fontsize=15)
    ax2.set_title('Distribution of '+ '%s' %var +' - Non-Defaulters', fontsize=15)
    plt.show()

In [ ]:
biplot(application_data, 'NAME_CONTRACT_TYPE', False)

<span style="color:Blue"><b>Observation: </b>The percentage of Cash loans is way higher than the number of Revolving loans for both Target = 0 and Target = 1</span>

In [ ]:
biplot(application_data, 'CODE_GENDER',False)

<span style="color:Blue"><b>Observations: </b>the percentage of Females taking loans is higher than the number of Males for both Target = 0 and Target = 1</span>

In [ ]:
biplot(application_data, 'FLAG_OWN_REALTY',False)

<span style="color:Blue"><b>Observations: </b>most people applying for loan own realty</span>

In [ ]:
biplot(application_data, 'FLAG_OWN_CAR', False)

<span style="color:Blue"><b>Observations: </b>most of the people applying for loan do not own a car. people with car are more in case of non-defaulters</span>

In [ ]:
biplot(application_data, 'NAME_HOUSING_TYPE', True)

<span style="color:Blue"><b>Observations: </b>Most people live in House/Apartment People living with parents have higher chance of having payment difficulties.</span>

In [ ]:
biplot(application_data, 'NAME_FAMILY_STATUS', True)

<span style="color:Blue"><b>Observations: </b>ratio of single/unmarried people is more in defaulters. Single/ not married people are more likely to have defaulted.</span>

In [ ]:
biplot(application_data, 'NAME_EDUCATION_TYPE', True)

<span style="color:Blue"><b>Observations:</b>People with higher education are non-defaulters, they are people with lesser risk as they are less likely of difficulties in payment</span>

In [ ]:
biplot(application_data, 'NAME_INCOME_TYPE', True)

<span style="color:Blue"><b>Observations: </b>Commercial associates, Pensioner, State Servants have a higher ratio to total in non-defaulters.</span>

In [ ]:
#function for plotting categories based on defaulters and non-defaulters for organization type
def biplot_orgtype(data, var,label_rotation):
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(40,15))
    s1=sns.countplot(ax=ax1,x=defaulters[var], data=defaulters, order= defaulters[var].value_counts().index)
    if(label_rotation):
        s1.set_xticklabels(s1.get_xticklabels(),rotation=90,fontsize=18)
    ax1.set_title('Distribution of '+ '%s' %var +' - Defaulters', fontsize=15)
    ax1.set_xlabel('%s' %var,fontsize=15)
    ax1.set_ylabel("Count of Loans",fontsize=15)
   
    s2=sns.countplot(ax=ax2,x=nondefaulters[var], data=nondefaulters, order= nondefaulters[var].value_counts().index)
    if(label_rotation):
        s2.set_xticklabels(s2.get_xticklabels(),rotation=90,fontsize=18)
    ax2.set_xlabel('%s' %var, fontsize=15)
    ax2.set_ylabel("Count of Loans", fontsize=15)
    ax2.set_title('Distribution of '+ '%s' %var +' - Non-Defaulters', fontsize=15)
    plt.show()

In [ ]:
biplot_orgtype(application_data, 'ORGANIZATION_TYPE', True)

<span style="color:Blue"><b>Observations: </b>Applicants working in 'Business Entity type 3' seems to be the highest both in defaulter and non-defaulter category.</span>

In [ ]:
 biplot(application_data, 'OCCUPATION_TYPE', True)

<span style="color:Blue"><b>Observations:</b> Applicants whose Occupation types are Laborers or Sales staff seems to be the highest both in defaulter and non-defaulter category.</span>

In [ ]:
application_data['AMT_INCOME_TOTAL'].describe()

In [ ]:
defaulters.loc[:,'INCOME_BRACKET']=pd.qcut(application_data.loc[:,'AMT_INCOME_TOTAL'],q=[0,0.10,0.35,0.50,0.90,1], labels=['Very_low','Low','Medium','High','Very_high'])
nondefaulters.loc[:,'INCOME_BRACKET']=pd.qcut(application_data.loc[:,'AMT_INCOME_TOTAL'],q=[0,0.10,0.35,0.50,0.90,1], labels=['Very_low','Low','Medium','High','Very_high'])

In [ ]:
biplot(application_data, 'INCOME_BRACKET', True)

<span style="color:Blue"><b>Observations: </b>Defaulters had very low total income as compared to nondefaulters</span>

###### Finding the int64 and float64 columns

In [ ]:
application_data.select_dtypes('int64').columns

In [ ]:
application_data.select_dtypes('float64').columns

###### Removing outliers for the numerical columns

In [ ]:
def outlier_compare_charts(var, percentile):
   
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15,8))
    s1=sns.boxplot(ax=ax1,y=application_data[var],data=application_data,x=application_data['TARGET'])
    s1.set_title("With_Outliers", fontsize=10)
    s1.set_ylabel('%s' %var)
    
    s2=sns.boxplot(ax=ax2,x=application_data['TARGET'],y=application_data[application_data[var]<np.nanpercentile(application_data[var], percentile)][var]);
    s2.set_title(var + ' boxplot on data within '+ str(percentile) +' percentile');
    s2.set_ylabel('%s' %var)

In [ ]:
outlier_compare_charts('AMT_INCOME_TOTAL', 95)

<span style="color:Blue"><b>Inferences:</b> Most applicants with payment difficulty have incomes in the lower range between 100000 to 200000.</span>

In [ ]:
#update dataframes
defaulters=application_data[application_data.TARGET==1] 
nondefaulters=application_data[application_data.TARGET==0]

In [ ]:
outlier_compare_charts('AMT_CREDIT', 99)

<span style="color:Blue"><b> Inference:</b> Most Defaulters have taken Loan credits in a lower range </span>

In [ ]:
#update dataframes
defaulters=application_data[application_data.TARGET==1] 
nondefaulters=application_data[application_data.TARGET==0]

In [ ]:
outlier_compare_charts('AMT_ANNUITY', 99)

<span style="color:Blue"><b>Inference: </b>Most Defaulter's Annuities are also very low in the range of a max of 30000</span>

In [ ]:
#Finding outliers in No.Of Children the applicants have
outlier_compare_charts('CNT_CHILDREN', 75)

<span style="color:Blue"><b>Inference:</b> Only 25% of the defaulters have one or more children</span>

In [ ]:
#update dataframes
defaulters=application_data[application_data.TARGET==1] 
nondefaulters=application_data[application_data.TARGET==0]

In [ ]:
#function for histogram
def plothist(var):
    plt.figure(figsize=(15,5))
    s=plt.hist(x=defaulters[var]);
    plt.xlabel('%s' %var)
    plt.ylabel("Count of Loans")
    plt.title('Histogram of '+ '%s' %var +' for Defaulters', fontsize=10)
plt.show()

In [ ]:
# converting the days of birth to age
application_data['AGE'] = application_data['DAYS_BIRTH'] / 365

In [ ]:
plt.hist(application_data['AGE']);

plt.title('Age in years.');

<span style="color:Blue"><b>Observation: </b>age is evenly distributed</span>

In [ ]:
#update dataframes
defaulters=application_data[application_data.TARGET==1] 
nondefaulters=application_data[application_data.TARGET==0]

In [ ]:
application_data['DAYS_EMPLOYED'].describe()

<span style="color:Blue"><b>Observation : </b>DAYS_EMPLOYED column has the value 365243 for pensioners. This data creates a weird anomaly when there are many negtive numbers indicating the actual number of days the applicants have been employed, so we can see the outliers for this field without including the pensioners which gives a real idea of how many days the applicants on an average are employed</span>

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
sns.boxplot(ax=axes[0,1],y=-1*(defaulters[defaulters.DAYS_EMPLOYED!=365243].DAYS_EMPLOYED)).set_title("Without Pensioner's DAYS_EMPLOYED for defaulters")
sns.boxplot(ax=axes[0,0],y=-1*(defaulters.DAYS_EMPLOYED)).set_title("With Pensioner's DAYS_EMPLOYED for Defaulters")
sns.boxplot(ax=axes[1,1],y=-1*(nondefaulters[nondefaulters.DAYS_EMPLOYED!=365243].DAYS_EMPLOYED)).set_title("Without Pensioner's DAYS_EMPLOYED for nondefaulters")
sns.boxplot(ax=axes[1,0],y=-1*(nondefaulters.DAYS_EMPLOYED)).set_title("With Pensioner's DAYS_EMPLOYED for nondefaulters")

plt.show()

<span style="color:Blue"><b></b></span>

In [ ]:
#update dataframes
defaulters=application_data[application_data.TARGET==1] 
nondefaulters=application_data[application_data.TARGET==0]

###### Bivariate Analysis

In [ ]:
#Bivariate Analysis on AMT_GOODS_PRICE and AMT_CREDIT
fig, (ax1,ax2) = plt.subplots(ncols=2,figsize=(20,7))
sns.scatterplot(ax=ax1,x=defaulters['AMT_GOODS_PRICE'],y=defaulters['AMT_CREDIT'],color='red')
ax1.set_xlabel('Loan given for the Price of the Goods')
ax1.set_ylabel('Credit Amount of the loan')

sns.scatterplot(ax=ax2,x=nondefaulters['AMT_GOODS_PRICE'],y=nondefaulters['AMT_CREDIT'],color='purple')
ax2.set_xlabel('Loan given for the Price of the Goods')
ax2.set_ylabel('Credit Amount of the loan')
plt.show()

<span style="color:Blue"><b>Observations: </b>For a few nondefaulters, there were cases where the Price of the goods didnot match the credit amount of the loan.</span>

In [ ]:
#Convert income into thousands to make an easy display 35000 will be 35 thousands
application_data['AMT_INCOME_TOTAL_IN_THOUSANDS'] =application_data['AMT_INCOME_TOTAL'] //1000

In [ ]:
# Remove the outlier for CNT_CHILDREN
application_data = application_data[application_data.CNT_CHILDREN < 8]
# Remove the outlier for AMT_CREDIT
application_data = application_data[application_data.AMT_INCOME_TOTAL_IN_THOUSANDS < 1500000]

In [ ]:
#update dataframes
defaulters=application_data[application_data.TARGET==1] 
nondefaulters=application_data[application_data.TARGET==0]

###### Bivariate Analysis

In [ ]:
def bivariateplots(var1,var2):
    fig, (ax1,ax2) = plt.subplots(ncols=2,figsize=(20,7))
    sns.scatterplot(ax=ax1,x=defaulters[var1],y=defaulters[var2],color='red')
    ax1.set_xlabel(var1)
    ax1.set_ylabel(var2)
    ax1.set_title("Defaulters")
    sns.scatterplot(ax=ax2,x=nondefaulters[var1],y=nondefaulters[var2],color='blue')
    ax2.set_xlabel(var1)
    ax2.set_ylabel(var2)
    ax2.set_title("Non Defaulters")
    plt.show()

In [ ]:
#Bivariate Analysis on AMT_GOODS_PRICE and AMT_CREDIT
bivariateplots('AMT_GOODS_PRICE','AMT_CREDIT')

<span style="color:Blue">
    <b>Observation : </b>For a few nondefaulters, there were cases where the Price of the goods didnot match the credit amount of the loan.
</span>

In [ ]:
#Convert income into thousands to make an easy displa 35000 will be 35 thousands
application_data['AMT_INCOME_TOTAL_IN_THOUSANDS'] =application_data['AMT_INCOME_TOTAL'] //1000

In [ ]:
# Remove the outlier for CNT_CHILDREN
application_data = application_data[application_data.CNT_CHILDREN < 8]
# Remove the outlier for AMT_CREDIT
application_data = application_data[application_data.AMT_INCOME_TOTAL_IN_THOUSANDS < 1500000]

In [ ]:
#update dataframes
defaulters=application_data[application_data.TARGET==1] 
nondefaulters=application_data[application_data.TARGET==0]

In [ ]:
#Bivariate Analysis on Children Count and Income total
bivariateplots('CNT_CHILDREN','AMT_INCOME_TOTAL_IN_THOUSANDS')

<span style="color:Blue">
<b>Observation : </b>Most of the applicants (both Defaulters and Non Defaulters) Total income was below 20000000 irrespective of the number of children they had.
</span>

In [ ]:
#Bivariate Analysis on Children Count and Credit Amount
bivariateplots('CNT_CHILDREN','AMT_CREDIT')

<span style="color:Blue"><b>Observation : </b>The above plot clearly shows that the No.of children the applicant had, can have a negative impact on the Credit Amount that was provided.</span>

In [ ]:
#Bivariate Analysis on Total Income and Credit Amount
bivariateplots('AMT_INCOME_TOTAL_IN_THOUSANDS','AMT_CREDIT')

<span style="color:Blue"><b>Observation : </b>The above plot clearly shows that the credit amount was variable irrespective of the Income</span>

In [ ]:
#Mutivariate Analysis on Weekday process started  and Credit Amount
bivariateplots('WEEKDAY_APPR_PROCESS_START','AMT_CREDIT')

<span style="color:Blue"><b>Observation : </b>The above plot clearly shows that the credit amount was variable irrespective of the day the application was made</span>

In [ ]:
#binning ages
age_data = application_data.loc[:,['TARGET', 'DAYS_BIRTH']]
age_data.loc[:,'YEARS_BIRTH'] = application_data.loc[:,'DAYS_BIRTH']/ 365
# Bin the age data
age_data.loc[:,'YEARS_BINNED'] = pd.cut(age_data.loc[:,'YEARS_BIRTH'], bins = np.linspace(20, 70, num = 11))
age_data.head()

In [ ]:
age_groups  = age_data.groupby('YEARS_BINNED').mean()
age_groups

In [ ]:
plt.figure(figsize = (8, 8))

# Graph the age bins and the average of the target as a bar plot
plt.bar(age_groups.index.astype(str), 100 * age_groups['TARGET'])

# Plot labeling
plt.xticks(rotation = 75); plt.xlabel('Age Group (years)'); plt.ylabel('Failure to Repay (%)')
plt.title('Failure to Repay by Age Group');

<span style="color:Blue"><b>
    Observation : </b>The Risk of Failure to repay was higher in the younger age group as compared to older applicants. As the age increased the risk decreased
</span>


In [ ]:
#Columns for Correlation_Maps
cols=['EXT_SOURCE_1',
      'CNT_CHILDREN',
      'AMT_INCOME_TOTAL',
      'AMT_CREDIT',
      'AMT_ANNUITY',
      'AMT_GOODS_PRICE',
      'HOUR_APPR_PROCESS_START',
      'DAYS_BIRTH',
      'DAYS_EMPLOYED',
      'CNT_FAM_MEMBERS',
      'AMT_REQ_CREDIT_BUREAU_QRT',
      'REGION_RATING_CLIENT_W_CITY', 
      'REGION_RATING_CLIENT',
       'DAYS_ID_PUBLISH',]

In [ ]:
defaulters_1=defaulters[cols]
defaulters_correlation = defaulters_1.corr()
round(defaulters_correlation, 3)

In [ ]:
# figure size
plt.figure(figsize=(20,15))

# heatmap
sns.heatmap(defaulters_correlation, cmap="coolwarm", annot=True)
plt.show()

<span style="color:Blue"><b>Observation:</b>   
    5 most positive correlations are-
    <ol>
      <li>AMT_CREDIT - AMT_GOODS_PRICE</li>
      <li>REGION_RATING_CLIENT_W_CITY - REGION_RATING_CLIENT</li>
      <li>CNT_CHILDREN - CNT_FAM_MEMBERS</li>
      <li>AMT_CREDIT - AMT_ANNUITY</li>
      <li>AMT_GOODS_PRICE - AMT_ANNUITY</li>
   </ol>
</span>

<span style="color:Blue"><b>Observation:</b>   
    5 most negative correlations are-
    <ol>
      <li>HOUR_APPR_PROCESS_START - REGION_RATING_CLIENT_W_CITY</li>
      <li>REGION_RATING_CLIENT - HOUR_APPR_PROCESS_START</li>
      <li>EXT_SOURCE_1 - DAYS_BIRTH</li>
      <li>DAYS_BIRTH - EXT_SOURCE_1</li>
      <li>DAYS_BIRTH - DAYS_EMPLOYED</li>
   </ol>
</span>

In [ ]:
nondefaulters_1=nondefaulters[cols]
nondefaulters_correlation = nondefaulters_1.corr()
round(nondefaulters_correlation, 3)

In [ ]:
# figure size
plt.figure(figsize=(20,15))

# heatmap
sns.heatmap(nondefaulters_correlation, cmap="coolwarm", annot=True)
plt.show()

<span style="color:Blue"><b>Observation:</b>   
    5 most positive correlations are-
    <ol>
      <li>AMT_CREDIT - AMT_GOODS_PRICE</li>
      <li>REGION_RATING_CLIENT_W_CITY - REGION_RATING_CLIENT</li>
      <li>CNT_CHILDREN - CNT_FAM_MEMBERS</li>
      <li>AMT_CREDIT - AMT_ANNUITY</li>
      <li>AMT_GOODS_PRICE - AMT_ANNUITY</li>
   </ol>
</span>

<span style="color:Blue"><b>Observation:</b>   
    5 most positive correlations are-
    <ol>
      <li>HOUR_APPR_PROCESS_START - REGION_RATING_CLIENT_W_CITY</li>
      <li>REGION_RATING_CLIENT - HOUR_APPR_PROCESS_START</li>
      <li>EXT_SOURCE_1 - DAYS_BIRTH</li>
      <li>DAYS_BIRTH - EXT_SOURCE_1</li>
      <li>DAYS_BIRTH - DAYS_EMPLOYED</li>
   </ol>
</span>


<span style="color:Brown">Analysis of Previous_Application : the data that shows Client's previous loan data</span>

In [ ]:
#importing data from CSV file into pandas dataframe
previous_data = pd.read_csv('/kaggle/input/credit-card/previous_application.csv')

In [ ]:
previous_data.shape

In [ ]:
previous_data.info(verbose=True)

In [ ]:
previous_data.head()

In [ ]:
previous_data['DAYS_DECISION'] = abs(previous_data['DAYS_DECISION'])
previous_data['SELLERPLACE_AREA'] = abs(previous_data['SELLERPLACE_AREA'])
previous_data['DAYS_FIRST_DUE'] = abs(previous_data['DAYS_FIRST_DUE'])
previous_data['DAYS_LAST_DUE_1ST_VERSION'] = abs(previous_data['DAYS_LAST_DUE_1ST_VERSION'])
previous_data['DAYS_LAST_DUE'] = abs(previous_data['DAYS_LAST_DUE'])
previous_data['DAYS_TERMINATION'] = abs(previous_data['DAYS_TERMINATION'])

In [ ]:
previous_data.dtypes.value_counts()

In [ ]:
previous_data.describe()

In [ ]:
# Get the count,size and unique values in each column of previous application data
application_data.agg(['count','size','nunique'])

In [ ]:
previous_data.NAME_CONTRACT_STATUS.unique()

In [ ]:
sns.barplot(x=previous_data[previous_data.NAME_CONTRACT_STATUS!='XNA'].NAME_CONTRACT_STATUS.value_counts().index,y=100*previous_data[previous_data.NAME_CONTRACT_STATUS!='XNA'].NAME_CONTRACT_STATUS.value_counts(normalize=True))
plt.xlabel("Contract Status", fontsize=12)
plt.ylabel("Count of Contract Status", fontsize=12)
plt.title("Distribution of Contract Status")
plt.show()

<span style="color:Blue"><b>Observations: </b>most of the contracts are approved</span>

In [ ]:
total_null = previous_data.isnull().sum().sort_values(ascending=False)
total_null

In [ ]:
percentage_null = (previous_data.isnull().sum()/previous_data.isnull().count()*100).sort_values(ascending=False)

percentage_null

In [ ]:
#drop the columns with 60 % or more null values
nullcol_60_previous = list(percentage_null[percentage_null.values>=60.0].index)
previous_data.drop(labels=nullcol_60_previous,axis=1,inplace=True)

In [ ]:
previous_data.info()

In [ ]:
# Listing down columns which are not needed
Unnecessary_previous = ['NFLAG_INSURED_ON_APPROVAL',
                        'DAYS_FIRST_DUE',
                        'DAYS_LAST_DUE_1ST_VERSION',
                        'WEEKDAY_APPR_PROCESS_START','HOUR_APPR_PROCESS_START',
                        'FLAG_LAST_APPL_PER_CONTRACT','NFLAG_LAST_APPL_IN_DAY']  

In [ ]:
previous_data.drop(labels=Unnecessary_previous, axis=1,inplace=True)
previous_data.shape

In [ ]:
previous_data.info()

In [ ]:
previous_data.nunique().sort_values()

In [ ]:
previous_data.head()

In [ ]:
#there are negative values for DAYS_DECISION , we need to make them positive
previous_data['DAYS_DECISION']=abs(previous_data['DAYS_DECISION'])

In [ ]:
#Group days taken to decide on the application outcome
previous_data['DAYS_DECISION_GROUP'] = (previous_data['DAYS_DECISION']-(previous_data['DAYS_DECISION'] % 400)).astype(str)+'-'+ ((previous_data['DAYS_DECISION'] - (previous_data['DAYS_DECISION'] % 400)) + (previous_data['DAYS_DECISION'] % 400) + (400 - (previous_data['DAYS_DECISION'] % 400))).astype(str)

In [ ]:
count= previous_data['DAYS_DECISION_GROUP'].value_counts(normalize=True)

In [ ]:
plt.figure(figsize=(10,5))
ax = sns.countplot(data=previous_data, x='DAYS_DECISION_GROUP', order=previous_data['DAYS_DECISION_GROUP'].value_counts().index,palette="muted")
ax.set_title("DAYS_DECISION")
plt.show()

<span style="color:Blue"><b>Observation: </b>Mostly decision was made within an year</span>

In [ ]:
#current data types and count of them
previous_data.dtypes.value_counts()

In [ ]:
approved=previous_data[previous_data.NAME_CONTRACT_STATUS=='Approved']
refused=previous_data[previous_data.NAME_CONTRACT_STATUS=='Refused']
canceled=previous_data[previous_data.NAME_CONTRACT_STATUS=='Canceled']
unused=previous_data[previous_data.NAME_CONTRACT_STATUS=='Unused offer']

In [ ]:
percentage_approved=(len(approved)*100)/len(previous_data)
percentage_refused=(len(refused)*100)/len(previous_data)
percentage_canceled=(len(canceled)*100)/len(previous_data)
percentage_unused=(len(unused)*100)/len(previous_data)

In [ ]:
print("The Percentage of people whose loans have been Approved:",round(percentage_approved,2))
print("The Percentage of people whose loans have been Refused:",round(percentage_refused,2))
print("The Percentage of people whose loans have been Canceled:",round(percentage_canceled,2))
print("The Percentage of people whose loans have been Unused:",round(percentage_unused,2))

In [ ]:
previous_data.apply(lambda x : x =='XNA').sum().sort_values(ascending=False)

###### Univariate analysis on variables categorised on the outcome of the loan application

In [ ]:
def quad_charts(var, label_rotation):
   
    fig, (ax1, ax2, ax3,ax4) = plt.subplots(ncols=4, figsize=(20,5))
    fig.suptitle("Univariate analysis on "+ var +" categorised on the outcome of the loan application")
    #sns.barplot(ax=axes[1] ,x=applications.CODE_GENDER.value_counts().index,y=applications.CODE_GENDER.value_counts(normalize=True)).set_title("GENDER_SPLIT")

    s1=sns.barplot(ax=ax1,x=refused[var].value_counts().index, data=refused, y= 100.* refused[var].value_counts(normalize=True))
    ax1.set_title("Refused (%)", fontsize=10)
    ax1.set_xlabel('%s' %var)
    ax1.set_ylabel("Count of Loans(in Percentage of Total)")
    if(label_rotation):
        s1.set_xticklabels(s1.get_xticklabels(),rotation=90)
    
    s2=sns.barplot(ax=ax2,x=approved[var].value_counts().index, data=approved, y= 100.* approved[var].value_counts(normalize=True))
    #s2=sns.countplot(ax=ax2,x=approved[var], data=approved, order= approved[var].value_counts().index,)
    if(label_rotation):
        s2.set_xticklabels(s2.get_xticklabels(),rotation=90)
    ax2.set_xlabel('%s' %var)
    ax2.set_ylabel("Count of Loans(in Percentage of Total)")
    ax2.set_title("Approved(%)", fontsize=10)
    
    s3=sns.barplot(ax=ax3,x=canceled[var].value_counts().index, data=canceled, y= 100.* canceled[var].value_counts(normalize=True))
    #s3=sns.countplot(ax=ax3,x=canceled[var], data=canceled, order= canceled[var].value_counts().index,)
    ax3.set_title("Canceled(%)", fontsize=10)
    ax3.set_xlabel('%s' %var)
    ax3.set_ylabel("Count of Loans(in Percentage of Total)")
    if(label_rotation):
        s3.set_xticklabels(s3.get_xticklabels(),rotation=90)
       
    s4=sns.barplot(ax=ax4,x=unused[var].value_counts().index, data=unused, y= 100.* unused[var].value_counts(normalize=True))
    #s4=sns.countplot(ax=ax4,x=unused[var], data=unused, order= unused[var].value_counts().index,)
    ax4.set_title("Unused offer(%)", fontsize=10)
    ax4.set_xlabel('%s' %var)
    ax4.set_ylabel("Count of Loans(in Percentage of Total)")
    if(label_rotation):
        s4.set_xticklabels(s4.get_xticklabels(),rotation=90)
    
    plt.show()

In [ ]:
quad_charts('PRODUCT_COMBINATION', True)

<span style="color:Blue"><b>Observation: </b>mostly, the loans approved for POS household with interest loans which were cancelled were Cash Loans for refused loans, max was of Cash X-Sell</span>

In [ ]:
#drop xna values in NAME_YIELD_GROUP
approved = approved.drop(approved[approved.NAME_YIELD_GROUP=='XNA'].index)
refused = refused.drop(refused[refused.NAME_YIELD_GROUP=='XNA'].index)
canceled = canceled.drop(canceled[canceled.NAME_YIELD_GROUP=='XNA'].index)
unused = unused.drop(unused[unused.NAME_YIELD_GROUP=='XNA'].index)

In [ ]:
quad_charts('NAME_YIELD_GROUP', True)

<span style="color:Blue"><b>Observations: </b>rejected loans are mostly from yield group not specified(XNA) approved loans are mostly from middle yield group</span>

In [ ]:
quad_charts('NAME_PORTFOLIO', True)

<span style="color:Blue"><b>Observations: </b>most rejected loans were in Cash most approved loans were POS</span>

In [ ]:
quad_charts('CHANNEL_TYPE', True)

<span style="color:Blue"><b>Observations: </b>most of the refused loans were in credit and cash offices channel most of the approved loans were in country-wide channel</span>

In [ ]:
previous_data = previous_data.drop(previous_data[previous_data.NAME_PRODUCT_TYPE=='XNA'].index)

In [ ]:
quad_charts('NAME_PRODUCT_TYPE', True)

<span style="color:Blue"><b>Observations: </b>most of the refused loans were in x-Sell for approved loans, the product-type are not specified</span>

In [ ]:
quad_charts('NAME_PAYMENT_TYPE', True)

<span style="color:Blue"><b>Observations: </b>for both previous as well as approved loans, loans were payed back in cash through the bank</span>

In [ ]:
quad_charts('NAME_CONTRACT_TYPE', False)

<span style="color:Blue"><b>Observations: </b>for refused loans, mostly the product type is cash loan approved loans were mostly consumer loans</span>

In [ ]:
quad_charts('NAME_CLIENT_TYPE', False)

<span style="color:Blue"><b>Observations: </b>for both approved and refused loans clients are repeaters</span>

###### Outliers and removal of outliers on

In [ ]:
def outlier_compare_charts(var, percentile):
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(20,5))

    s1=sns.boxplot(ax=ax1,y=var,data=previous_data,x='NAME_CONTRACT_STATUS')
    s1.set_title("With_Outliers", fontsize=10)
    s1.set_ylabel('%s' %var)

    s2=sns.boxplot(ax=ax2,data=previous_data,x='NAME_CONTRACT_STATUS',y=previous_data[previous_data[var]<np.nanpercentile(previous_data[var], percentile)][var]);    
    s2.set_title(var + ' boxplot on data within '+ str(percentile) +' percentile');
    s2.set_ylabel('%s' %var)
    
    plt.show()

In [ ]:
outlier_compare_charts('AMT_ANNUITY',90)

<span style="color:Blue"><b>Observations: </b>Amount of Annuity has outliers on top 10 percent(on left) which has been removed (on right)</span>

In [ ]:
outlier_compare_charts('AMT_CREDIT',80)

<span style="color:Blue"><b>Observations: </b>Amount of Credit issued in the Previous Applications has outliers on top 20 percent (on left) which has been removed (on right)</span>

In [ ]:
outlier_compare_charts('AMT_GOODS_PRICE',90)

<span style="color:Blue"><b>Observations: </b>Amount of Goods Price has outliers on top 10 percent (on left) which has been removed (on right)</span>

In [ ]:
outlier_compare_charts('AMT_APPLICATION',80)

<span style="color:Blue"><b>Observations: </b>Application Loan amount in the Previous Applications has outliers on top 20 percent (on left) which has been removed (on right)</span>

###### Bivariate Analysis

In [ ]:
#Bivariate Analysis on Price of the Goods and Credit Amount
fig, (ax1,ax2,ax3,ax4) = plt.subplots(ncols=4,figsize=(20,10))
s1=sns.scatterplot(ax=ax1,x=approved['AMT_GOODS_PRICE'],y=approved['AMT_CREDIT'],color='red')
ax1.set_xlabel('Price of the Goods')
ax1.set_ylabel('Credit Amount')
ax1.set_title("Approved", fontsize=10)

s2=sns.scatterplot(ax=ax2,x=refused['AMT_GOODS_PRICE'],y=refused['AMT_CREDIT'],color='red')
ax2.set_xlabel('Price of the Goods')
ax2.set_ylabel('Credit Amount')
ax2.set_title("Refused", fontsize=10)

s3=sns.scatterplot(ax=ax3,x=canceled['AMT_GOODS_PRICE'],y=canceled['AMT_CREDIT'],color='red')
ax3.set_xlabel('Price of the Goods')
ax3.set_ylabel('Credit Amount')
ax3.set_title("Canceled", fontsize=10)

s4=sns.scatterplot(ax=ax4,x=unused['AMT_GOODS_PRICE'],y=unused['AMT_CREDIT'],color='red')
ax4.set_xlabel('Price of the Goods')
ax4.set_ylabel('Credit Amount')
ax4.set_title("Unused Offers", fontsize=10)

plt.show()

<span style="color:Blue"><b>Observations: </b>correlation between the Price of the Goods and Amount of Credit provided seems linear except for a few applications</span>

In [ ]:
#Bivariate Analysis on Amount Load applied for and Credit Amount
fig, (ax1,ax2,ax3,ax4) = plt.subplots(ncols=4,figsize=(20,10))
s1=sns.scatterplot(ax=ax1,x=approved['AMT_APPLICATION'],y=approved['AMT_CREDIT'],color='blue')
ax1.set_xlabel('Amount Loans Applied for')
ax1.set_ylabel('Credit Amount')
ax1.set_title("Approved", fontsize=10)

s2=sns.scatterplot(ax=ax2,x=refused['AMT_APPLICATION'],y=refused['AMT_CREDIT'],color='blue')
ax2.set_xlabel('Amount Loans Applied for')
ax2.set_ylabel('Credit Amount')
ax2.set_title("Refused", fontsize=10)

s3=sns.scatterplot(ax=ax3,x=canceled['AMT_APPLICATION'],y=canceled['AMT_CREDIT'],color='blue')
ax3.set_xlabel('Amount Loans Applied for')
ax3.set_ylabel('Credit Amount')
ax3.set_title("Canceled", fontsize=10)

s4=sns.scatterplot(ax=ax4,x=unused['AMT_APPLICATION'],y=unused['AMT_CREDIT'],color='blue')
ax4.set_xlabel('Amount Loans Applied for')
ax4.set_ylabel('Credit Amount')
ax4.set_title("Unused Offers", fontsize=10)

plt.show()

<span style="color:Blue"><b>Observations: </b>correlation between the Amount Loan applied for and Amount of Credit provided seems linear except for a few applications which means the variation between the Loan amount requested has been the credit provided.</span>

In [ ]:
col_approved=['AMT_GOODS_PRICE', 'AMT_ANNUITY', 
               'CNT_PAYMENT', 'AMT_CREDIT', 'DAYS_DECISION', 'AMT_APPLICATION','DAYS_TERMINATION', 
              'DAYS_LAST_DUE','DAYS_FIRST_DRAWING', 'AMT_GOODS_PRICE']

In [ ]:
col_refused=['AMT_GOODS_PRICE', 'AMT_ANNUITY', 
               'CNT_PAYMENT', 'AMT_CREDIT', 'DAYS_DECISION', 'AMT_APPLICATION']

In [ ]:
approved_num_col=approved[col_approved]

In [ ]:
refused_num_col=refused[col_refused]

In [ ]:
#calculating correlation for approved
approved_correlation = approved_num_col.corr()
round(approved_correlation, 3)

In [ ]:
c1=approved_correlation.unstack()
c1.sort_values(ascending=False).drop_duplicates().head(10)

In [ ]:
c1=approved_correlation.unstack()
c1.sort_values(ascending=False).drop_duplicates().tail(10)

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(approved_correlation, cmap='YlGnBu', annot=True)
plt.show()

<span style="color:Blue">Observations: This Correlation Map confirms the high correlation between the variables AMT_CREDIT,AMT_APPLICATION,AMT_GOODS_PRICE and AMT_ANNUITY thus confirming that these are very much dependent on each other. The correlation between these Amounts shows a negative impact on the number of days it takes for the decision to be made.</span>

<span style="color:Blue"><b>Observation:</b>   
    5 most positive correlations are-
    <ol>
      <li>AMT_CREDIT - AMT_APPLICATION</li>
      <li>AMT_GOODS_PRICE - AMT_ANNUITY</li>
      <li>AMT_ANNUITY - AMT_CREDIT</li>
      <li>AMT_ANNUITY -  AMT_APPLICATION</li>
      <li>CNT_PAYMENT - AMT_CREDIT</li>
   </ol>
</span>

<span style="color:Blue"><b>Observation:</b>   
    5 most negative correlations are-
    <ol>
      <li>AMT_CREDIT - DAYS_DECISION</li>
      <li>AMT_APPLICATION - DAYS_DECISION</li>
      <li>AMT_ANNUITY - DAYS_DECISION</li>
      <li>CNT_PAYMENT - DAYS_DECISION</li>
      <li>AMT_GOODS_PRICE - DAYS_DECISION</li>
   </ol>
</span>

In [ ]:
#calculating correlation for refused
refused_correlation = refused_num_col.corr()
round(refused_correlation, 3)

In [ ]:
c1=refused_correlation.unstack()
c1.sort_values(ascending=False).drop_duplicates().head(10)

In [ ]:
c1=refused_correlation.unstack()
c1.sort_values(ascending=False).drop_duplicates().tail(10)

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(refused_correlation, cmap='coolwarm', annot=True)
plt.show()

<span style="color:Blue"> Observations: This Correlation Map confirms the high correlation between the variables AMT_CREDIT,AMT_APPLICATION,AMT_GOODS_PRICE and AMT_ANNUITY thus confirming that these are very much dependent on each other. The correlation between these Amounts shows a negative impact on the number of days it takes for the decision to be made.</span>

<span style="color:Blue"><b>Observation:</b>   
    5 most positive correlations are-
    <ol>
      <li>AMT_CREDIT - AMT_APPLICATION</li>
      <li>AMT_GOODS_PRICE - AMT_ANNUITY</li>
      <li>AMT_ANNUITY - AMT_CREDIT</li>
      <li>AMT_ANNUITY -  AMT_APPLICATION</li>
      <li>CNT_PAYMENT - AMT_CREDIT</li>
   </ol>
</span>

<span style="color:Blue"><b>Observation:</b>   
    5 most negative correlations are-
   <ol>
      <li>AMT_CREDIT - DAYS_DECISION</li>
      <li>AMT_APPLICATION - DAYS_DECISION</li>
      <li>AMT_ANNUITY - DAYS_DECISION</li>
      <li>CNT_PAYMENT - DAYS_DECISION</li>
      <li>AMT_GOODS_PRICE - DAYS_DECISION</li>
   </ol>
</span>

<span style="color:Blue">Observations: Finally the above correlation maps show that the Loan Amounts have no impact on the decision made as approved or rejected</span>